In [1]:
import spacy
from spacy import displacy
from spacy.tokens import Span
import pandas as pd

In [32]:
nlp = spacy.load('en_core_web_sm')

In [33]:
# Import the Matcher library
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [2]:
myfile = open('eligibilities.txt')

In [3]:
df = pd.read_csv('eligibilities.txt', sep='|')

In [9]:
df.head()

,id,nct_id,sampling_method,gender,minimum_age,maximum_age,healthy_volunteers,population,criteria,gender_description,gender_based
0,4287621,NCT04464980,NaN,All,18 Years,NaN,No,NaN,~ Inclusion Criteria for Retention Phase...,NaN,NaN
1,4287622,NCT04464967,NaN,All,18 Years,75 Years,No,NaN,~ Inclusion Criteria:~ - Capab...,NaN,NaN
2,4295848,NCT04357873,NaN,All,18 Years,NaN,No,NaN,~ Inclusion Criteria:~ 1. Aged ...,NaN,NaN
3,4293842,NCT04384016,NaN,All,12 Months,12 Years,Accepts Healthy Volunteers,NaN,~ Inclusion Criteria:~ - Child...,NaN,NaN
4,4299025,NCT04316442,NaN,All,18 Years,NaN,No,NaN,~ Inclusion Criteria:~ 1. Confi...,NaN,NaN


In [14]:
df[['nct_id','criteria']].head()

,nct_id,criteria
0,NCT04464980,~ Inclusion Criteria for Retention Phase...
1,NCT04464967,~ Inclusion Criteria:~ - Capab...
2,NCT04357873,~ Inclusion Criteria:~ 1. Aged ...
3,NCT04384016,~ Inclusion Criteria:~ - Child...
4,NCT04316442,~ Inclusion Criteria:~ 1. Confi...


In [15]:
df.shape

(349610, 11)

In [16]:
df_sample = df[:5000]

In [17]:
# df_sample = df_sample.dropna()
df_sample.shape

(5000, 11)

In [18]:
df_sample.columns

Index(['id', 'nct_id', 'sampling_method', 'gender', 'minimum_age',
       'maximum_age', 'healthy_volunteers', 'population', 'criteria',
       'gender_description', 'gender_based'],
      dtype='object')

In [19]:
criteria = df_sample[['criteria','nct_id']]

In [20]:
criteria = criteria.dropna()
criteria.shape

(4989, 2)

In [21]:
def extract_criteria_text(text):
    text =text.replace('~',' ')
    text =text.replace('  ',' ')
    inc_ind = text.lower().find('inclusion criteria')
    exc_ind = text.lower().find('exclusion criteria')
    inc_cri = text[inc_ind:exc_ind]
    exc_cri = text[exc_ind:-1]
    return([inc_cri,exc_cri])
    

In [22]:
def extract_criteria(sample_criteria):

#     if(type(sample_criteria)==float):
#         print(sample_criteria)
        
    
    sample_criteria =sample_criteria.replace('~',' ')
    sample_criteria =sample_criteria.replace('  ',' ')
    doc = nlp.make_doc(sample_criteria)
    

    # pattern1 = [{"LIKE_NUM": True}, {"TEXT": "."}]
    pattern2 = [{"LOWER":"inclusion"},{"LOWER":"criteria"}]
    pattern3 = [{"LOWER":"exclusion"},{"LOWER":"criteria"}]
    # matcher.add('startWord', None, pattern1)
    # matcher.remove('startWord')
    matcher.add('topicWord', None, pattern2,pattern3)
    
    matches = matcher(doc)
#     for matchid,start,end in matches:
#         print(matchid,start,end)
    inc_start = 0
    inc_end = 0
    exc_start = 0
    exc_end = 0

    if len(matches) == 2:

        match_id,start,end = matches[0]
        inc_start = end + 1

        match_id,start,end = matches[1]
        inc_end = start - 1
        exc_start = end + 1
        exc_end = len(doc)

#     print(inc_start, inc_end, exc_start, exc_end)
    inclusion_criteria = Span(doc, inc_start, inc_end)
    exclusion_criteria = Span(doc, exc_start, exc_end)
#     print(inclusion_criteria.text)
#     print(exclusion_criteria.text)
    return([inclusion_criteria,exclusion_criteria])

In [23]:
#sample_criteria = criteria[5]

In [24]:
#extracted_criteria = extract_criteria(sample_criteria)


extracted_criterias = list(map(extract_criteria,criteria['criteria']))

In [25]:
extracted_criterias = list(map(extract_criteria_text,criteria['criteria']))

In [26]:
type(extracted_criterias)
len(extracted_criterias)

4989

In [27]:
criteria_df = pd.DataFrame(extracted_criterias,columns=['inclusion','exclusion'])

In [28]:
print(criteria_df.shape)
print(criteria.shape)

(4989, 2)
(4989, 2)


In [29]:
new_df = pd.concat([criteria['nct_id'] , criteria_df],axis = 1)

In [30]:
new_df.shape

(5000, 3)

In [31]:
new_df.to_csv('extracted_criteria.csv',sep = '|')

In [32]:
new_df.head()

,nct_id,inclusion,exclusion
0,NCT04464980,Inclusion Criteria for Retention Phase: Indivi...,Exclusion Criteria for Retention Phase: Indivi...
1,NCT04464967,Inclusion Criteria: - Capable of giving s...,Exclusion Criteria: - Pregnant and/or lac...
2,NCT04357873,Inclusion Criteria: 1. Aged ≥18 years old...,Exclusion Criteria: 1. Prior treatment wi...
3,NCT04384016,Inclusion Criteria: - Children from full ...,Exclusion Criteria: - Hypersensitivity re...
4,NCT04316442,Inclusion Criteria: 1. Confirmed diagnosi...,Exclusion Criteria: 1. Isolated vascular ...
